# Mine Cricketer's Data from ESPNCRICINFO

In [1]:
import requests

In [2]:
import bs4

In [3]:
base_url = 'http://stats.espncricinfo.com/ci/engine/player/'

In [4]:
params = [
    ('class', '3'),
    ('template', 'results'),
    ('type', 'batting'),
    ('view', 'match')
]

In [5]:
player_id = '28235'

In [6]:
url = base_url + player_id + '.html'

In [7]:
response = requests.get(url=url, params=params)

In [8]:
print(response.status_code)

200


In [9]:
print(response.text)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<!-- hostname: web001, edition-view: espncricinfo-en-in, country: unknown, cluster: www, created: 2019-04-14 22:03:33 -->
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://developers.facebook.com/schema/" >
<head>
 <script type="text/javascript">var _sf_startpt=(new Date()).getTime()</script>
 <meta name="google-site-verification" content="ZxdgH3XglRg0Bsy-Ho2RnO3EE4nRs53FloLS6fkt_nc" />
 <title>Batting records | Twenty20 Internationals | Cricinfo Statsguru | ESPNcricinfo.com</title>
 <meta http-equiv="Content-Type" content="text/html;charset=utf-8" />
 <meta name="keywords" content="" />
 
 <meta name="description" content="" />
<!--[if IE 9]>
<script language="javascript" type="text/javascript">
function fnCreateJumpList(iScenario) {
fnClearJumpList();
window.ext

In [10]:
soup = bs4.BeautifulSoup(response.text,'lxml')

In [11]:
table = soup.find_all("table")[3]

In [12]:
print(table)

<table class="engineTable">
<caption>Match by match list</caption>
<thead>
<tr class="headlinks">
<th class="padAst" nowrap=""><a class="black-link" href="/ci/engine/player/28235.html?class=3;orderby=batting_score1;template=results;type=batting;view=match" title="sort by runs scored in 1st innings">Bat1</a></th>
<th nowrap=""><a class="black-link" href="/ci/engine/player/28235.html?class=3;orderby=runs;template=results;type=batting;view=match" title="sort by runs scored">Runs</a></th>
<th nowrap=""><a class="black-link" href="/ci/engine/player/28235.html?class=3;orderby=balls_faced;template=results;type=batting;view=match" title="sort by balls faced">BF</a></th>
<th nowrap=""><a class="black-link" href="/ci/engine/player/28235.html?class=3;orderby=batting_strike_rate;template=results;type=batting;view=match" title="sort by batting strike rate">SR</a></th>
<th nowrap=""><a class="black-link" href="/ci/engine/player/28235.html?class=3;orderby=fours;template=results;type=batting;view=matc

In [13]:
rows = bs4.BeautifulSoup(str(table))

In [14]:
match_data = rows.find_all('tr')[1:]

In [15]:
data = []
i = 0
for match in match_data:
    i = i+1
    isDismissed = False
    didNotBat = False

    cols = bs4.BeautifulSoup(str(match))
    
    score_cell = cols.find_all('td')[0]
    soup = bs4.BeautifulSoup(str(score_cell))
    score = soup.get_text()
    if score == "DNB" or score == "TDNB":
        didNotBat = True
        runs = 0
    elif "*" in score:
        runs = int(score[:-1])
    else:
        isDismissed = True
        runs =  int(score)
        

    bf_cell = cols.find_all('td')[2]
    soup = bs4.BeautifulSoup(str(bf_cell))
    balls = soup.get_text()
    if balls == "-":
        ball = 0
        strikeRate = 0
    else:
        ball = balls
        strikeRate = round(int(runs)/int(balls) * 100,2)
        
    fours_cell  = cols.find_all('td')[4]
    soup = bs4.BeautifulSoup(str(fours_cell))
    fours = soup.get_text()
    if fours == "-":
        four = 0
    else:
        four = fours
    
    sixes_cell  = cols.find_all('td')[5]
    soup = bs4.BeautifulSoup(str(sixes_cell))
    sixes = soup.get_text()
    if sixes == "-":
        six = 0
    else:
        six = sixes
        
    
    opposition_cell  = cols.find_all('td')[7]
    soup = bs4.BeautifulSoup(str(opposition_cell))
    opposition = soup.get_text()[2:]
    
    ground_cell  = cols.find_all('td')[8]
    soup = bs4.BeautifulSoup(str(ground_cell))
    ground = soup.get_text()
    
    date_cell  = cols.find_all('td')[9]
    soup = bs4.BeautifulSoup(str(date_cell))
    date = soup.get_text()
    
    id_cell  = cols.find_all('td')[10]
    soup = bs4.BeautifulSoup(str(id_cell))
    matchID = soup.get_text().split("#",1)[1] 
    
    entry = {
        'balls': int(ball),
        'batPosition': None,
        'captained': False,
        'date': date,
        'didNotBat': didNotBat,
        'dismissalType': None,
        'dismissedAgainst': None,
        'dismissalBowlerHand': None,
        'matchID': int(matchID),
        'minutes': 0,
        'fours': int(four),
        'firstBat': False,
        'ground': ground,
        'innings': 1,
        'match': int(i),
        'notOut': not isDismissed,
        'opposition': opposition,
        'runs': int(runs),
        'sixes': int(six),
        'strikeRate': int(strikeRate),
        'won': False
    }
    data.append(entry)

In [16]:
import pandas as pd

In [17]:
dataset = pd.DataFrame(data)
print(dataset)

    balls batPosition  captained         date  didNotBat dismissalBowlerHand  \
0      11        None      False   4 Jun 2011      False                None   
1      19        None      False  10 Oct 2013      False                None   
2      28        None      False  21 Mar 2014      False                None   
3       3        None      False  23 Mar 2014      False                None   
4       6        None      False  28 Mar 2014      False                None   
5      28        None      False   7 Sep 2014      False                None   
6       4        None      False   2 Oct 2015      False                None   
7      12        None      False   5 Oct 2015      False                None   
8       8        None      False  26 Jan 2016      False                None   
9      32        None      False  29 Jan 2016      False                None   
10      9        None      False  31 Jan 2016      False                None   
11     13        None      False   9 Feb

In [18]:
params = [
    ('class', '3'),
    ('template', 'results'),
    ('type', 'batting'),
    ('view', 'match'),
    ('result','1')
]
response = requests.get(url=url, params=params)
soup = bs4.BeautifulSoup(response.text,'lxml')
table = soup.find_all("table")[3]
match_data = table.find_all('tr')[1:]

In [19]:
won = []
for match in match_data:
    id_cell  = match.find_all('td')[10]
    soup = bs4.BeautifulSoup(str(id_cell))
    matchID = soup.get_text().split("#",1)[1]
    won.append(int(matchID))
print(won)

[200, 331, 378, 382, 389, 485, 486, 489, 497, 499, 509, 515, 517, 521, 541, 547, 553, 623, 630, 632, 652, 655, 657, 659, 660, 662, 678, 684, 690, 707, 709, 710, 714, 737]


In [20]:
params = [
    ('captain','1'),
    ('class', '3'),
    ('filter','advanced'),
    ('template', 'results'),
    ('type', 'batting'),
    ('view', 'match'),
]
response = requests.get(url=url, params=params)
soup = bs4.BeautifulSoup(response.text,'lxml')
table = soup.find_all("table")[3]
match_data = table.find_all('tr')[1:]

In [21]:
captained = []
for match in match_data:
    id_cell  = match.find_all('td')[10]
    soup = bs4.BeautifulSoup(str(id_cell))
    matchID = soup.get_text().split("#",1)[1]
    captained.append(int(matchID))
print(captained)

IndexError: list index out of range

In [22]:
params = [
    ('class', '3'),
    ('filter','advanced'),
    ('template', 'results'),
    ('type', 'batting'),
    ('view', 'innings'),
]
response = requests.get(url=url, params=params)
soup = bs4.BeautifulSoup(response.text,'lxml')
table = soup.find_all("table")[3]
match_data = table.find_all('tr')[1:]

batPosition = []
batFirst = []
dismissalType = []
minutes = []
for match in match_data:

    id_cell  = match.find_all('td')[13]
    soup = bs4.BeautifulSoup(str(id_cell))
    matchID = soup.get_text().split("#",1)[1]

    position_cell  = match.find_all('td')[6]
    soup = bs4.BeautifulSoup(str(position_cell))
    batPosition.append({
        "id": int(matchID),
        "pos": soup.get_text()
    })
    
    minutes_cell  = match.find_all('td')[1]
    soup = bs4.BeautifulSoup(str(minutes_cell))
    if soup.get_text() != '-':
        minutes.append({
            "id": int(matchID),
            "min": soup.get_text()
        })
    
    dismissal_cell  = match.find_all('td')[7]
    soup = bs4.BeautifulSoup(str(dismissal_cell))
    if soup.get_text() != '-' or soup.get_text() != 'not out':
        dismissalType.append({
            "id": int(matchID),
            "type": soup.get_text()
        })
    
    innings_cell  = match.find_all('td')[8]
    soup = bs4.BeautifulSoup(str(innings_cell))
    if soup.get_text() == '1':  
        batFirst.append(matchID)

In [23]:
dismissedAgainst = []

params = [
    ('class', '3'),
    ('bowling_pacespin','1'),
    ('filter','advanced'),
    ('template', 'results'),
    ('type', 'batting'),
    ('view', 'dismissal_list'),
]
response = requests.get(url=url, params=params)
soup = bs4.BeautifulSoup(response.text,'lxml')
table = soup.find_all("table")[3]
match_data = table.find_all('tr')[1:]

for match in match_data:

    id_cell  = match.find_all('td')[9]
    soup = bs4.BeautifulSoup(str(id_cell))
    matchID = soup.get_text().split("#",1)[1]
    dismissedAgainst.append({
        "type": "pace",
        "id": int(matchID)
    })
    
params = [
    ('class', '3'),
    ('bowling_pacespin','2'),
    ('filter','advanced'),
    ('template', 'results'),
    ('type', 'batting'),
    ('view', 'dismissal_list'),
]
response = requests.get(url=url, params=params)
soup = bs4.BeautifulSoup(response.text,'lxml')
table = soup.find_all("table")[3]
match_data = table.find_all('tr')[1:]

for match in match_data:

    id_cell  = match.find_all('td')[9]
    soup = bs4.BeautifulSoup(str(id_cell))
    matchID = soup.get_text().split("#",1)[1]
    dismissedAgainst.append({
        "type": "spin",
        "id": int(matchID)
    })
    
print(dismissedAgainst)


[{'type': 'pace', 'id': 200}, {'type': 'pace', 'id': 378}, {'type': 'pace', 'id': 389}, {'type': 'pace', 'id': 405}, {'type': 'pace', 'id': 457}, {'type': 'pace', 'id': 485}, {'type': 'pace', 'id': 489}, {'type': 'pace', 'id': 496}, {'type': 'pace', 'id': 497}, {'type': 'pace', 'id': 509}, {'type': 'pace', 'id': 515}, {'type': 'pace', 'id': 521}, {'type': 'pace', 'id': 541}, {'type': 'pace', 'id': 553}, {'type': 'pace', 'id': 624}, {'type': 'pace', 'id': 631}, {'type': 'pace', 'id': 632}, {'type': 'pace', 'id': 652}, {'type': 'pace', 'id': 657}, {'type': 'pace', 'id': 660}, {'type': 'pace', 'id': 678}, {'type': 'pace', 'id': 684}, {'type': 'pace', 'id': 690}, {'type': 'pace', 'id': 707}, {'type': 'pace', 'id': 712}, {'type': 'pace', 'id': 714}, {'type': 'pace', 'id': 735}, {'type': 'pace', 'id': 737}, {'type': 'pace', 'id': 749}, {'type': 'spin', 'id': 331}, {'type': 'spin', 'id': 382}, {'type': 'spin', 'id': 486}, {'type': 'spin', 'id': 535}, {'type': 'spin', 'id': 547}, {'type': 'spi

In [24]:
dismissalBowlerHand = []

params = [
    ('class', '3'),
    ('bowling_hand','1'),
    ('filter','advanced'),
    ('template', 'results'),
    ('type', 'batting'),
    ('view', 'dismissal_list'),
]
response = requests.get(url=url, params=params)
soup = bs4.BeautifulSoup(response.text,'lxml')
table = soup.find_all("table")[3]
match_data = table.find_all('tr')[1:]

for match in match_data:

    id_cell  = match.find_all('td')[9]
    soup = bs4.BeautifulSoup(str(id_cell))
    matchID = soup.get_text().split("#",1)[1]
    dismissalBowlerHand.append({
        "hand": "right",
        "id": int(matchID)
    })
    
params = [
    ('class', '3'),
    ('bowling_hand','2'),
    ('filter','advanced'),
    ('template', 'results'),
    ('type', 'batting'),
    ('view', 'dismissal_list'),
]
response = requests.get(url=url, params=params)
soup = bs4.BeautifulSoup(response.text,'lxml')
table = soup.find_all("table")[3]
match_data = table.find_all('tr')[1:]

for match in match_data:

    id_cell  = match.find_all('td')[9]
    soup = bs4.BeautifulSoup(str(id_cell))
    matchID = soup.get_text().split("#",1)[1]
    dismissalBowlerHand.append({
        "hand": "left",
        "id": int(matchID)
    })
    
print(dismissalBowlerHand)

[{'hand': 'right', 'id': 200}, {'hand': 'right', 'id': 378}, {'hand': 'right', 'id': 382}, {'hand': 'right', 'id': 389}, {'hand': 'right', 'id': 405}, {'hand': 'right', 'id': 457}, {'hand': 'right', 'id': 485}, {'hand': 'right', 'id': 486}, {'hand': 'right', 'id': 489}, {'hand': 'right', 'id': 496}, {'hand': 'right', 'id': 497}, {'hand': 'right', 'id': 509}, {'hand': 'right', 'id': 515}, {'hand': 'right', 'id': 521}, {'hand': 'right', 'id': 535}, {'hand': 'right', 'id': 541}, {'hand': 'right', 'id': 553}, {'hand': 'right', 'id': 630}, {'hand': 'right', 'id': 632}, {'hand': 'right', 'id': 652}, {'hand': 'right', 'id': 654}, {'hand': 'right', 'id': 656}, {'hand': 'right', 'id': 657}, {'hand': 'right', 'id': 659}, {'hand': 'right', 'id': 660}, {'hand': 'right', 'id': 678}, {'hand': 'right', 'id': 707}, {'hand': 'right', 'id': 712}, {'hand': 'right', 'id': 735}, {'hand': 'right', 'id': 737}, {'hand': 'left', 'id': 331}, {'hand': 'left', 'id': 547}, {'hand': 'left', 'id': 624}, {'hand': 'le

In [26]:
for match in won:
    dataset.loc[dataset['matchID'] == match, 'won'] = True
for match in captained:
    dataset.loc[dataset['matchID'] == match, 'captained'] = True
for match in dismissalType:
    dataset.loc[dataset['matchID'] == match['id'], 'dismissalType'] = match['type']
for match in dismissedAgainst:
    dataset.loc[dataset['matchID'] == match['id'], 'dismissedAgainst'] = match['type']
for match in dismissalBowlerHand:
    dataset.loc[dataset['matchID'] == match['id'], 'dismissalBowlerHand'] = match['hand']
for match in minutes:
    dataset.loc[dataset['matchID'] == match['id'], 'minutes'] = match['min']
for match in batPosition:
    dataset.loc[dataset['matchID'] == match['id'], 'batPosition'] = match['pos']
for match in batFirst:
    dataset.loc[dataset['matchID'] == match, 'firstBat'] = True

In [27]:
print(dataset)

    balls batPosition  captained         date  didNotBat dismissalBowlerHand  \
0      11           2      False   4 Jun 2011      False               right   
1      19           2      False  10 Oct 2013      False                left   
2      28           2      False  21 Mar 2014      False               right   
3       3           2      False  23 Mar 2014      False               right   
4       6           2      False  28 Mar 2014      False               right   
5      28           2      False   7 Sep 2014      False               right   
6       4           2      False   2 Oct 2015      False                None   
7      12           2      False   5 Oct 2015      False               right   
8       8           2      False  26 Jan 2016      False               right   
9      32           2      False  29 Jan 2016      False               right   
10      9           2      False  31 Jan 2016      False               right   
11     13           2      False   9 Feb

In [28]:
json = dataset.to_json(orient='records')
print(json)

[{"balls":11,"batPosition":"2","captained":false,"date":"4 Jun 2011","didNotBat":false,"dismissalBowlerHand":"right","dismissalType":"caught","dismissedAgainst":"pace","firstBat":false,"fours":0,"ground":"Port of Spain","innings":1,"match":1,"matchID":200,"minutes":"13","notOut":false,"opposition":"West Indies","runs":5,"sixes":0,"strikeRate":45,"won":true},{"balls":19,"batPosition":"2","captained":false,"date":"10 Oct 2013","didNotBat":false,"dismissalBowlerHand":"left","dismissalType":"stumped","dismissedAgainst":"spin","firstBat":false,"fours":5,"ground":"Rajkot","innings":1,"match":2,"matchID":331,"minutes":"40","notOut":false,"opposition":"Australia","runs":32,"sixes":0,"strikeRate":168,"won":true},{"balls":28,"batPosition":"2","captained":false,"date":"21 Mar 2014","didNotBat":false,"dismissalBowlerHand":"right","dismissalType":"caught","dismissedAgainst":"pace","firstBat":false,"fours":5,"ground":"Dhaka","innings":1,"match":3,"matchID":378,"minutes":"35","notOut":false,"oppositi